In [1]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import random
import joblib

In [2]:
# Node labels
farms = ["F"+str(i) for i in range(1,11)]
custs = ["C"+str(i) for i in range(1,6)]
prods = ['P1', 'P2', 'P3']
dealer = "Dealer"

In [3]:
G = nx.DiGraph()

In [4]:
# Add nodes to network graph
G.add_nodes_from(farms)
G.add_node(dealer)
G.add_nodes_from(prods)
G.add_nodes_from(custs)

In [5]:
farm_dealer_edge_tuples =[(f,dealer) for f in farms]
dealer_prod_edge_tuples =[(dealer, p) for p in prods]
prod_cust_tuples = [(p,c) for p in prods for c in custs]

In [6]:
edges = farm_dealer_edge_tuples + dealer_prod_edge_tuples + prod_cust_tuples

In [7]:
# Connect nodes with edges
G.add_edges_from(edges)

In [8]:
# Add quantities supplied by farms
G['F1']['Dealer']['quantity']=100
G['F2']['Dealer']['quantity']=100
G['F3']['Dealer']['quantity']=100
G['F4']['Dealer']['quantity']=100
G['F5']['Dealer']['quantity']=100
G['F6']['Dealer']['quantity']=100
G['F7']['Dealer']['quantity']=100
G['F8']['Dealer']['quantity']=100
G['F9']['Dealer']['quantity']=100
G['F10']['Dealer']['quantity']=100

In [9]:
# Add prices per egg charged by farms 
G['F1']['Dealer']['cost_per_egg']=0.25
G['F2']['Dealer']['cost_per_egg']=0.25
G['F3']['Dealer']['cost_per_egg']=0.25
G['F4']['Dealer']['cost_per_egg']=0.25
G['F5']['Dealer']['cost_per_egg']=0.25
G['F6']['Dealer']['cost_per_egg']=0.25
G['F7']['Dealer']['cost_per_egg']=0.25
G['F8']['Dealer']['cost_per_egg']=0.25
G['F9']['Dealer']['cost_per_egg']=0.25
G['F10']['Dealer']['cost_per_egg']=0.25

In [10]:
# Set the type of product
G.nodes['P1']['eggs_per_box'] = 6
G.nodes['P2']['eggs_per_box'] = 10
G.nodes['P3']['eggs_per_box'] = 12

In [11]:
# Set the type of customer
G.nodes['C1']['category'] = 'hypermarket'
G.nodes['C2']['category'] = 'supermarket'
G.nodes['C3']['category'] = 'supermarket'
G.nodes['C4']['category'] = 'grocery_store'
G.nodes['C5']['category'] = 'grocery_store'

In [12]:
# Set the customer location
G.nodes['C1']['location'] = 'North'
G.nodes['C2']['location'] = 'South'
G.nodes['C3']['location'] = 'North'
G.nodes['C4']['location'] = 'North'
G.nodes['C5']['location'] = 'South'

In [13]:
# Set the transport costs per egg based on location
transport_cost_per_egg = {'North':0.10, 'South':0.15}

In [14]:
hypermarket_tupes = [(p,c) for (p,c) in prod_cust_tuples if G.nodes[c]['category']=='hypermarket']
supermarket_tupes = [(p,c) for (p,c) in prod_cust_tuples if G.nodes[c]['category']=='supermarket']
grocery_store_tupes = [(p,c) for (p,c) in prod_cust_tuples if G.nodes[c]['category']=='grocery_store']

In [15]:
# Demand
# Hypermarkets
G['P1']['C1']['demand'] = 200
G['P2']['C1']['demand'] = 100
G['P3']['C1']['demand'] = 50

# Supermarkets
G['P1']['C2']['demand'] = 50
G['P2']['C2']['demand'] = 25
G['P3']['C2']['demand'] = 15

G['P1']['C3']['demand'] = 50
G['P2']['C3']['demand'] = 25
G['P3']['C3']['demand'] = 15

# Grocery store
G['P1']['C4']['demand'] = 50
G['P2']['C4']['demand'] = 0
G['P3']['C4']['demand'] = 0

G['P1']['C5']['demand'] = 50
G['P2']['C5']['demand'] = 0
G['P3']['C5']['demand'] = 0

In [16]:
[(p, G.nodes[p]['eggs_per_box']) for p in prods]

[('P1', 6), ('P2', 10), ('P3', 12)]

In [17]:
# Set selling price per product
# Hypermarkets
G['P1']['C1']['price'] = 9
G['P2']['C1']['price'] = 15
G['P3']['C1']['price'] = 18

# Supermarkets
G['P1']['C2']['price'] = 10.5
G['P2']['C2']['price'] = 17.5
G['P3']['C2']['price'] = 21

G['P1']['C3']['price'] = 10.5
G['P2']['C3']['price'] = 17.5
G['P3']['C3']['price'] = 21

# Grocery store
G['P1']['C4']['price'] = 12
G['P2']['C4']['price'] = 0
G['P3']['C4']['price'] = 0

G['P1']['C5']['price'] = 12
G['P2']['C5']['price'] = 0
G['P3']['C5']['price'] = 0


In [18]:
# Save the graph as a joblib object
joblib.dump(G, 'supply_chain_graph')
joblib.dump(farms, 'farms_list')
joblib.dump(custs, 'customer_list')
joblib.dump(prods, 'product_list')
joblib.dump(transport_cost_per_egg, 'transport_costs')

['transport_costs']

Fill the hypermarket orders first 

In [19]:
category_order = ['hypermarket', 'supermarket', 'grocery_store']

In [155]:
ordered_tuples = [(p,cust) for co in category_order for p in prods for cust in G.successors(p) if G.nodes[cust]['category']==co]


In [37]:
supply = get_total_supply(G)
supply

1000

In [196]:
def fill_order(order_list):

    global prods, G, supply
    ordered_tuples = [(p,cust) for co in order_list for p in prods for cust in G.successors(p) if G.nodes[cust]['category']==co]
    supply_copy = supply.copy()
    zero_array = np.zeros(len(ordered_tuples))
    for i, ((p, c), z) in enumerate(zip(ordered_tuples, zero_array)):
        if G[p][c]['demand'] ==0:
            continue
        else:
            if supply_copy > G.nodes[p]['eggs_per_box']:
                total_supply_boxes = int(supply_copy/G.nodes[p]['eggs_per_box'])
                if total_supply_boxes > G[p][c]['demand']:
                    zero_array[i] = G[p][c]['demand']
                    supply_copy -= G[p][c]['demand']*G.nodes[p]['eggs_per_box']
                elif total_supply_boxes < G[p][c]['demand']:
                    zero_array[i] = total_supply_boxes
                    supply_copy -= total_supply_boxes * G.nodes[p]['eggs_per_box']
            else:
                break
    return zero_array              


In [200]:
q_vec = fill_order(category_order)
q_vec

array([166.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.])

In [238]:
def calc_profit(qty_vec, order_list, graph):
    global prods, G, transport_cost_per_egg
    ordered_tuples = [(p,cust) for co in order_list for p in prods for cust in G.successors(p) if G.nodes[cust]['category']==co]
    avg_cost_per_egg = np.mean([graph[farm]['Dealer']['cost_per_egg'] for farm in graph.predecessors('Dealer')])
    total_eggs = np.sum([qv * graph.nodes[p]['eggs_per_box'] for (p,c), qv  in zip(ordered_tuples, qty_vec)])
    total_egg_cost = avg_cost_per_egg * total_eggs 
    total_sales = np.sum([G[p][c]['price']*qv for (p,c), qv in zip(ordered_tuples, qty_vec)])
    transport_cost = np.sum([transport_cost_per_egg[graph.nodes[c]['location']] * graph.nodes[p]['eggs_per_box'] * qv  for (p,c), qv in zip(ordered_tuples, qty_vec)])
    return total_sales - (transport_cost+ total_egg_cost) 

In [239]:
calc_profit(qty_vec=q_vec, order_list=category_order, graph=G)

1145.4

In [226]:
transport_cost_per_egg[G.nodes['C1']['location']]

0.1

In [223]:
transport_cost_per_egg

{'North': 0.1, 'South': 0.15}

In [35]:
from pso import get_total_supply, demand, get_supply_costs, random_initiate_vec

In [26]:
def fill_order(prod_cust_tuple:tuple)-> int:
    global supply, prods, G
    prod_cap = np.array([graph.nodes[p]['eggs_per_box'] for p in prods])
    

In [75]:
[ (p,cust) for p in prods for cust in G.successors(p)]

[('P1', 'C1'),
 ('P1', 'C2'),
 ('P1', 'C3'),
 ('P1', 'C4'),
 ('P1', 'C5'),
 ('P2', 'C1'),
 ('P2', 'C2'),
 ('P2', 'C3'),
 ('P2', 'C4'),
 ('P2', 'C5'),
 ('P3', 'C1'),
 ('P3', 'C2'),
 ('P3', 'C3'),
 ('P3', 'C4'),
 ('P3', 'C5')]

In [79]:
d = {1:('P1', 'C1'), 2: ('P1', 'C2'), 3: ('P1', 'C3')}

In [86]:
for v in d.values():
    print(v)

('P1', 'C1')
('P1', 'C2')
('P1', 'C3')


3

In [29]:
for pro, cus in ordered_tuples:
    print(pro, cus, 'demand ->', G[pro][cus]['demand'], "pack ->",G.nodes[pro]['eggs_per_box'])

P1 C1 demand -> 200 pack -> 6
P2 C1 demand -> 100 pack -> 10
P3 C1 demand -> 50 pack -> 12
P1 C2 demand -> 50 pack -> 6
P1 C3 demand -> 50 pack -> 6
P2 C2 demand -> 25 pack -> 10
P2 C3 demand -> 25 pack -> 10
P3 C2 demand -> 15 pack -> 12
P3 C3 demand -> 15 pack -> 12
P1 C4 demand -> 50 pack -> 6
P1 C5 demand -> 50 pack -> 6
P2 C4 demand -> 0 pack -> 10
P2 C5 demand -> 0 pack -> 10
P3 C4 demand -> 0 pack -> 12
P3 C5 demand -> 0 pack -> 12
